In [1]:
# Import required modules
from concurrent.futures import ProcessPoolExecutor
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from itertools import chain
import time
import urllib3
import glob
import os

In [2]:
# Suppress warnings
urllib3.disable_warnings()

# Define deaders and date
HEADERS = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36"}
today = pd.to_datetime("today").strftime("%d_%b_%y")


# This function generates cover page links
def generateCoverPageLink(url):
    """This function generates cover page links from total courses,
    url = url to make request in which total course number is found,
    return = cover pages links"""
    
    # Store cover page links
    coverPageLink = []
    
    # Making request
    r = requests.get(url, headers=HEADERS, verify=False)
    s = BeautifulSoup(r.text, "lxml")
    
    # Scrape total course number
    totalCourse = int(s.find("span", class_="h1").text.strip().replace(",", ""))
    
    # Create stop page
    stopPage = int(np.ceil(totalCourse/100))
    
    # Iterate through stop page and create cover page links
    for page in range(1, stopPage+1):
            coverPageLink.append(url + f"?pageno={page}&sortby=MostPopular&pagesize=100")
    return coverPageLink



# This function scrapes individual course links from every cover page links
def scrapeIndividualCourseLink(url):
    """Scrapes individual course link from cover page links,
    url = cover page link,
    return = Individual course links"""
    
    # Store course links
    courseLink = []
    
    # Making request
    r = requests.get(url, headers=HEADERS, verify=False)
    s = BeautifulSoup(r.text, "lxml")
    
    # Scrape course links and store
    for lnk in s.find_all("div", class_="course-overview"):
        courseLink.append("https://www.reed.co.uk" + lnk.find("h2").find("a").get("href"))
    return courseLink



# This function scrapes individual course info from individual course link
def scrapeCourseInfo(url):
    """url = individual course link,
    return = scraped course info as a dataframe"""
    
    # Initialize empty list of variables to be scraped
    courseTitle = []
    subtitle = []
    offerPrice = []
    originalPrice = []
    courseProvider = []
    unitSold = []
    category = []
    savings = []
    haveCpd = []
    awardingBody = []
    qualName = []
    cpdPoint = []
    isRegulated = []
    hasProfCert = []
    soldOrEnq = []
    
    # Making request
    r = requests.get(url, headers=HEADERS, verify=False)
    s = BeautifulSoup(r.text, "lxml")
    
    # Scrape course name
    try:
        courseTitle.append(s.find("div", class_="course-title").find("h1").text.strip())
    except:
        courseTitle.append("na")
        
    # Scrape subtitle
    try:
        subtitle.append(s.find("div", class_="course-title").find("h2").text.strip())
    except:
        subtitle.append("na")
        
    # Scrape offer price
    try:
        offerPrice.append(s.find("span", class_="current-price").text.strip())
    except:
        offerPrice.append("na")
        
    # Scrape original price
    try:
        originalPrice.append(s.find("small", class_="vat-status").text.strip())
    except:
        originalPrice.append("na")
        
    # Scrape course provider
    try:
        try:
            # If the provider is hyperlinked
            courseProvider.append(s.find("a", class_="provider-link").text.strip())
        except:
            # If the provider is not hyperlinked
            courseProvider.append(s.find("span", class_="thumbnail").text.strip())
    except:
        courseProvider.append("na")
        
    # Scrape unit sale
    try:
        unitSold.append(s.find(id="number-enquiries-purchases").text.strip())
    except:
        unitSold.append(0)
        
    # Scrape category
    try:
        # Scrape total category
        totalCat = len(s.find_all("ol", class_="breadcrumb pb-0"))
        for cat in range(totalCat):
            category.append([x.text.strip() for x in s.find_all("ol", class_="breadcrumb pb-0")[cat].find_all("li")])
    except:
        category.append("na")
        
    # Scrape savings
    try:
        savings.append(s.find("span", class_="icon-savings-tag price-saving").text.strip())
    except:
        savings.append("na")
        
    # Does the course have CPD?
    try:
        haveCpd.append(1 if s.find("div", class_="badge badge-dark badge-cpd mt-2") else 0)
    except:
        haveCpd.append(0)
    
    # Scrape awarding body
    try:
        try:
            # Executes if the course is "Endorsed by"
            awardingBody.append(s.find("div", class_="col").find("a").text.strip())
        except:
            # Executes if the course is "Awarded by"
            awardingBody.append(s.find("div", class_="small").find("div").find("a").text.strip())
    except:
        awardingBody.append("na")
            
    # Scrape qualification name, only for awarded courses
    try:
        qualName.append(s.find("div", class_="small").find("h3", class_="h4").text.strip())
    except:
        qualName.append("na")
        
    # Scrape cpd point
    try:
        cpdPoint.append(s.body.find_all(text=re.compile("\d{1,3}\sCPD hours / points"))[0].strip())
    except:
        cpdPoint.append(0)
        
    # Is the course regulated? Assign 1 if regulated, otherwise 0
    try:
        isRegulated.append(1 if s.find("div", class_="badge badge-dark badge-regulated mt-2") else 0)
    except:
        isRegulated.append(0)
        
    # Does the course offer professional certification?
    try:
        hasProfCert.append(1 if s.find("div", class_="badge badge-dark badge-professional mt-2") else 0)
    except:
        hasProfCert.append(0)
    
    # Check if the course is sold or enquired. 2 if the course has both purchased and enquired mode
    try:
        soldOrEnq.append(2 if (s.find(id="addToBasket") and s.find(id="enquireNow"))\
        else 1 if s.find(id="addToBasket") else 0)
    except:
        soldOrEnq.append("na")
        
    # Create a df off scraped variables
    df = pd.DataFrame({
        "courseTitle":courseTitle,
        "courseLink":url,
        "subtitle":subtitle,
        "courseProvider":courseProvider,
        "offerPrice":offerPrice,
        "originalPrice":originalPrice,
        "unitSold":unitSold,
        "category":[category], # This one is not scalar, converting into 1d
        "haveCpd":haveCpd,
        "cpdPoint":cpdPoint,
        "awardingBody":[awardingBody],
        "qualName":qualName,
        "isRegulated":isRegulated,
        "hasProfCert":hasProfCert,
        "savings":savings,
        "soldOrEnq":soldOrEnq
    })
    df = df.astype(str)
    return df



# This function cleans scraped data
def cleanAndExtractFeature(df):
    """df = dataFrame to clean,
    return = final cleaned data"""
    
    # Copy the input data
    finalDf = df.copy()
    
    # Create course id and insert to the df
    finalDf.insert(loc=0, value=finalDf.courseLink.str.split("/").str.get(5).str.replace("#",""), 
                     column="courseId")
    
    # Insert date
    finalDf.insert(loc=0, value=today, column="date")
    
    # Clean unit sold
    finalDf.unitSold = finalDf.unitSold.apply(lambda x: re.findall(r"\d+", x)).str.join("")
    finalDf.unitSold = pd.to_numeric(finalDf.unitSold, errors="coerce").fillna(0).astype("int")
    
    # Clean saving percent
    finalDf["savingsPercent"] = finalDf.savings.str.split("Save").str[-1].str.replace("%", "")
    finalDf.savingsPercent = pd.to_numeric(finalDf.savingsPercent, errors="coerce").fillna(0).astype("int")
    
    # Clean offer price
    finalDf.offerPrice = finalDf.offerPrice.str.split("£").str[-1].str.replace(",", "")
    
    # Clean original price
    finalDf.originalPrice = finalDf.originalPrice.str.split("£").str[-1].str.replace(",", "").str.replace(")", "")
    finalDf.originalPrice = pd.to_numeric(finalDf.originalPrice, errors="coerce")
    
    # If savings is 0, make offer price equals to original price
    finalDf.originalPrice = np.where(finalDf.savingsPercent==0,
                                     finalDf.originalPrice.fillna(finalDf.offerPrice), finalDf.originalPrice)
    
    # Extract CPD point
    finalDf["cpdPoint"] = pd.to_numeric(finalDf.cpdPoint.str.join("").str.split("CPD").str[0], errors="coerce").fillna(0).astype(int)
    
    # Clean CPD provider
    finalDf["cpdProvider"]  = finalDf.cpdProvider.str.join("").str.replace("Accredited by", "").str.strip()
    
    # Remove '"' from category
    finalDf.category = finalDf.category.apply(lambda x: eval(x))
    
    # Create broadCategory1 from category
    finalDf["broadCategory1"] = finalDf.category.str[0].str[0]
    
    # Create broadCategory2 from category
    finalDf["broadCategory2"] = finalDf.category.str[-1].str[0]
    
    # Create subCategory1 from category
    finalDf["subCategory1"] = finalDf.category.str[0].str[-1]
    
    # Create subCategory1 from category
    finalDf["subCategory2"] = finalDf.category.str[-1].str[-1]
    
    # Extract qualification name
    finalDf["qualName"] = np.where(finalDf.qualName.str.contains("CPD"), "na", finalDf.qualName)
    
    # Clean awarding body
    finalDf.awardingBody = finalDf.awardingBody.str[1:-1].str[1:-1]\
    .str.strip().replace(r"^\s*$", np.nan, regex=True).fillna("na")
    
    # Drop "savings"
    finalDf.drop("savings", axis=1, inplace=True)
    
    # Drop duplicates by "courseId"
    finalDf = finalDf.drop_duplicates("courseId")
    return finalDf

In [3]:
# Wrap all the functions inside main
def main1(url):
    """url = url to make the 1st requests to generate cover pages,
    return = final cleaned dataframe"""
    
    # Record start time
    startTime = time.time()
    
    # Generates cover pages links
    coverPageLink = generateCoverPageLink(url)
    
    # Store individual course links
    courseLink = []
    
    # This loop ensures maximum no of course links scraped
    for _ in range(6):
        with ProcessPoolExecutor(max_workers=6) as ex:
            # Scrape individual course links
            indCourseLink = list(ex.map(scrapeIndividualCourseLink, coverPageLink))
            indCourseLink = list(chain(*indCourseLink)) # Flattening the list
        courseLink.append(indCourseLink)
    courseLink = list(chain(*courseLink))
    
    # Create a series to drop duplicates by ids. This portion keeps only the unique links
    tempSeries = pd.Series(courseLink, name="tempLink")
    splitTempSeries = tempSeries.str.split("/", expand=True)
    splitTempSeries.columns = ["a","b","c","d","e","f","g","h"]
    duplicatesDropped = splitTempSeries.drop_duplicates("f", keep="first").reset_index(drop=True)
    courseLink = duplicatesDropped.agg("/".join, axis=1) # This returns a series of course links
    print(f"Total {courseLink.shape[0]} Courses will be Scraped.")
    return courseLink

ConnectionError: ('Connection aborted.', OSError(0, 'Error'))

In [ ]:
# Run the function to scrape course links
courseLink = main1("https://www.reed.co.uk/courses/discount")

# Save course link as xlsx
pd.DataFrame({"courseLink":courseLink}).to_excel(f"courseLink_{today}.xlsx", index=None)

# Read course links from the current directory
courseLink = pd.concat([pd.read_excel(f) for f in glob.glob(f"courseLink_{today}.xlsx")])

In [ ]:
# We will use function main2 to scrape course info by chunks (5)
def main2(chunkNo, chunkSize=5):
    """chunkNo = Index of the chunk to scrape,
    chunkSize = No. of chunks the course link will be splitted into (default=5)"""
    
    # Split course links into 5 chunks
    courseLinkSplitted = np.array_split(courseLink.courseLink.to_list(), chunkSize)
    
    # Scrapes course info from course link
    with ProcessPoolExecutor() as ex:
        courseInfo = pd.concat(list(ex.map(scrapeCourseInfo, courseLinkSplitted[chunkNo]))).reset_index(drop=True)
    
    # Cleans and engineers new features from the scraped dataframe and returns the final dataframe   
    finalDf = cleanAndExtractFeature(courseInfo).reset_index(drop=True)
    
    # Prints how many records scraped
    print(f"Finished! Scraped {finalDf.shape[0]} courses.")
    return finalDf

In [ ]:
%%time
# Scrape by chunks and save as excel
chunk0 = main2(0)
chunk0.to_excel(f"chunk0_{today}.xlsx", index=None)

In [ ]:
%%time
# Scrape by chunks
chunk1 = main2(1)
chunk1.to_excel(f"chunk1_{today}.xlsx", index=None)

In [ ]:
%%time
# Scrape by chunks
chunk2 = main2(2)
chunk2.to_excel(f"chunk2_{today}.xlsx", index=None)

In [ ]:
%%time
# Scrape by chunks
chunk3 = main2(3)
chunk3.to_excel(f"chunk3_{today}.xlsx", index=None)

In [ ]:
%%time
# Scrape by chunks
chunk4 = main2(4)
chunk4.to_excel(f"chunk4_{today}.xlsx", index=None)

In [22]:
# Read today's all the chunk files from the current dir
masterDf = pd.concat([pd.read_excel(f) for f in sorted(glob.glob(f"chunk*_{today}.xlsx"))]).reset_index(drop=True)
masterDf.head()

,date,courseId,courseTitle,courseLink,subtitle,courseProvider,offerPrice,originalPrice,unitSold,category,...,awardingBody,qualName,isRegulated,hasProfCert,soldOrEnq,savingsPercent,broadCategory1,broadCategory2,subCategory1,subCategory2
0,09_Apr_21,162300,Food Hygiene Course - Level 2,https://www.reed.co.uk/courses/food-hygiene-co...,"ROSPA Accredited, Institute of Hospitality End...",The Training Terminal,10.0,30.0,10075,"[['Food science', 'Food safety'], ['Health and...",...,Royal Society for the Prevention of Accidents,Level 2 Award,0,0,1,66,Food science,Hospitality & catering,Food safety,Food hygiene
1,09_Apr_21,266620,Leadership & Management,https://www.reed.co.uk/courses/leadership-mana...,Level 7 Advanced Diploma | 150 CPD Points |*FR...,CPD Courses,10.0,1050.0,8942,"[['HR', 'Leadership'], ['Management', 'Leaders...",...,The Quality Licence Scheme,na,0,0,1,99,HR,Management,Leadership,Leadership & management
2,09_Apr_21,227846,Microsoft Excel,https://www.reed.co.uk/courses/microsoft-excel...,Spring Sale! | FREE PDF Certificate | CPD Cert...,Janets,10.0,949.0,1758,"[['Office skills', 'Microsoft Office', 'Micros...",...,na,na,0,0,1,98,Office skills,Business,Microsoft Excel,Microsoft Excel
3,09_Apr_21,274566,Food Hygiene and Safety Level 3,https://www.reed.co.uk/courses/food-hygiene-an...,**Health Day Gifts:Free PDF Certificate+Free H...,Training Express Ltd,15.0,299.0,2246,"[['Health and safety', 'Food safety'], ['Hospi...",...,Institute of Hospitality,na,0,0,1,94,Health and safety,Hospitality & catering,Food safety,Food hygiene
4,09_Apr_21,98276,Project Management,https://www.reed.co.uk/courses/project-managem...,Advanced Diploma QLS Level 7 |*FREE PDF Certif...,Oxford Home Study College,10.0,1050.0,5296,"[['IT', 'Project management'], ['Project manag...",...,The Quality Licence Scheme,na,0,0,1,99,IT,Project management,Project management,PMI


In [23]:
# No of courses been scraped with total providers
masterDf.shape, masterDf.courseProvider.nunique()

((38219, 23), 209)

In [24]:
# Save the data as csv
masterDf.to_csv(f"/home/faysal/Desktop/masterData/allDiscountCourse/{today}_allDiscount.csv", index=None)

In [14]:
# Clean the directories by removing file not required any more
chunksToRemove = glob.glob(f"chunk*_{today}.xlsx")
linkToRemove = glob.glob(f"courseLink*_{today}.xlsx")

# Clean the directory. Remove links and chunks
[os.remove(f) for f in chunksToRemove]
[os.remove(f) for f in linkToRemove]

[None]